In [4]:
import pandas as pd
#read in the dataset
df = pd.read_csv('C:/Users/HP/Downloads/diabetes_csv.csv')
#take a look at the data
df.head()

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


In [6]:
df.shape

(768, 9)

In [7]:
#split data into inputs and targets
X = df.drop(columns = ['class'])
y = df['class']

In [8]:
X.head()

,preg,plas,pres,skin,insu,mass,pedi,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [9]:
y.head()

0    tested_positive
1    tested_negative
2    tested_positive
3    tested_negative
4    tested_positive
Name: class, dtype: object

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify = y)


In [13]:
X_train.shape

(614, 8)

In [14]:
X_test.shape

(154, 8)

In [15]:
y_train.shape

(614,)

# KNN model with the help of grid search cv algorithm 

In [22]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
#create gridsearchcv
knn_gsc = GridSearchCV(knn, params_knn, cv = 5)
knn_gsc.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [23]:
#save best model
knn_best = knn_gsc.best_estimator_
#check best n_neigbors value
print(knn_gsc.best_params_)

{'n_neighbors': 15}


# Random Forest model: it is itself an ensemble model

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
params_rf = {'n_estimators':[50, 100, 200]}
rf_gsc = GridSearchCV(rf,params_rf,cv = 5)
rf_gsc.fit(X_train,y_train)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [29]:
rf_best = rf_gsc.best_estimator_
print(rf_gsc.best_params_)

{'n_estimators': 100}


# Logistic regression model 

In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)

C:\Users\HP\AppData\Local\conda\conda\envs\tfp3.6\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Testing of all three models

In [35]:
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('lr: {}'.format(lr.score(X_test, y_test)))

knn: 0.7337662337662337
rf: 0.7402597402597403
lr: 0.7207792207792207


# creating ensemble model 

In [39]:
from sklearn.ensemble import VotingClassifier
estimator = [('knn', knn_best), ('rf', rf_best), ('log_reg', lr)]
ensemble = VotingClassifier(estimator,voting = 'hard')
ensemble.fit(X_train,y_train)
ensemble.score(X_test,y_test)

C:\Users\HP\AppData\Local\conda\conda\envs\tfp3.6\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7272727272727273